In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
import pandas as pd
from sklearn.utils import shuffle

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
X_train = X_train_full / 255.
X_test = X_test / 255.
y_train = y_train_full.astype(float)
y_test = y_test.astype(float)
X_train = X_train.reshape(-1, 28, 28, 1)

def build_model():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return model

In [4]:
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        
    return model

In [5]:
def k_fold(num_epochs):
    k = 10
    num_val = len(X_train) // k
    all_scores = []
    
    np.random.seed(42)
    tf.random.set_seed(42)
    
    for i in range(k):
        print('processing fold #', i)

        X_val = X_train[i * num_val: (i + 1) * num_val]
        y_val = y_train[i * num_val: (i + 1) * num_val]

        X_train_part = np.concatenate(
            [X_train[:i * num_val],
            X_train[(i + 1) * num_val:]],
            axis = 0)

        y_train_part = np.concatenate(
            [y_train[:i * num_val],
            y_train[(i + 1) * num_val:]],
            axis = 0)

        model = build_model()
        history = model.fit(X_train_part, y_train_part, 
                            epochs = num_epochs, verbose = 1)

        val_loss, val_acc = model.evaluate(X_val, y_val, verbose = 1)
        all_scores.append(val_acc)
        
    return np.mean(all_scores)

In [6]:
k_fold(100)

processing fold # 0
Epoch 1/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.5166 - accuracy: 0.8091
Epoch 2/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3295 - accuracy: 0.8798
Epoch 3/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2778 - accuracy: 0.8969
Epoch 4/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2481 - accuracy: 0.9077
Epoch 5/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2266 - accuracy: 0.9155
Epoch 6/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2055 - accuracy: 0.9226
Epoch 7/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1854 - accuracy: 0.9309
Epoch 8/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1709 - accuracy: 0.9352
Epoch 9/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1573 - accuracy: 0.9399
Epoch 10/100
1688/1688 [====================

1688/1688 [==============================] - 23s 14ms/step - loss: 0.0241 - accuracy: 0.9922
Epoch 77/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0239 - accuracy: 0.9925
Epoch 78/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0268 - accuracy: 0.9916
Epoch 79/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0264 - accuracy: 0.9917
Epoch 80/100
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0222 - accuracy: 0.9930
Epoch 81/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0225 - accuracy: 0.9930
Epoch 82/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0277 - accuracy: 0.9921
Epoch 83/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0196 - accuracy: 0.9940
Epoch 84/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0299 - accuracy: 0.9909
Epoch 85/100
1688/1688 [==============================] - 2

1688/1688 [==============================] - 24s 14ms/step - loss: 0.0291 - accuracy: 0.9901
Epoch 51/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0306 - accuracy: 0.9897
Epoch 52/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0334 - accuracy: 0.9886
Epoch 53/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0280 - accuracy: 0.9904
Epoch 54/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0322 - accuracy: 0.9896
Epoch 55/100
1688/1688 [==============================] - 23s 14ms/step - loss: 0.0261 - accuracy: 0.9911
Epoch 56/100
1688/1688 [==============================] - 23s 13ms/step - loss: 0.0327 - accuracy: 0.9892
Epoch 57/100
1688/1688 [==============================] - 23s 13ms/step - loss: 0.0280 - accuracy: 0.9905
Epoch 58/100
1688/1688 [==============================] - 23s 13ms/step - loss: 0.0271 - accuracy: 0.9910
Epoch 59/100
1688/1688 [==============================] - 2

1688/1688 [==============================] - 20s 12ms/step - loss: 0.0615 - accuracy: 0.9769
Epoch 26/100
1688/1688 [==============================] - 20s 12ms/step - loss: 0.0537 - accuracy: 0.9799
Epoch 27/100
1688/1688 [==============================] - ETA: 0s - loss: 0.0511 - accuracy: 0.98 - 20s 12ms/step - loss: 0.0511 - accuracy: 0.9808
Epoch 28/100
1688/1688 [==============================] - 20s 12ms/step - loss: 0.0546 - accuracy: 0.9791
Epoch 29/100
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0500 - accuracy: 0.9812
Epoch 30/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0468 - accuracy: 0.9824
Epoch 31/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0484 - accuracy: 0.9825
Epoch 32/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0443 - accuracy: 0.9839
Epoch 33/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0453 - accuracy: 0.9833
Epoch 34/100
1688

188/188 [==============================] - 1s 3ms/step - loss: 1.1794 - accuracy: 0.8972
processing fold # 3
Epoch 1/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.4979 - accuracy: 0.8156
Epoch 2/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3267 - accuracy: 0.8796
Epoch 3/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2785 - accuracy: 0.8982
Epoch 4/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2475 - accuracy: 0.9083
Epoch 5/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2226 - accuracy: 0.9171
Epoch 6/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2010 - accuracy: 0.92490s - loss: 0.2012 - accuracy: 
Epoch 7/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1811 - accuracy: 0.9321
Epoch 8/100
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1640 - accuracy: 0.9386
Epoch 9/100
1688/1688

...

1688/1688 [==============================] - 16s 9ms/step - loss: 0.0291 - accuracy: 0.9904
Epoch 58/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0270 - accuracy: 0.9910
Epoch 59/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0300 - accuracy: 0.9904
Epoch 60/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0237 - accuracy: 0.9925
Epoch 61/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0293 - accuracy: 0.9907
Epoch 62/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0305 - accuracy: 0.9906
Epoch 63/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0264 - accuracy: 0.9916
Epoch 64/100
1688/1688 [==============================] - 14s 9ms/step - loss: 0.0289 - accuracy: 0.9903
Epoch 65/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0302 - accuracy: 0.9908
Epoch 66/100
1688/1688 [==============================] - 15s 9ms/st

1688/1688 [==============================] - 15s 9ms/step - loss: 0.0383 - accuracy: 0.9864
Epoch 34/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0401 - accuracy: 0.9859
Epoch 35/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0367 - accuracy: 0.9865
Epoch 36/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0357 - accuracy: 0.9870
Epoch 37/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0399 - accuracy: 0.9870
Epoch 38/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0342 - accuracy: 0.9876
Epoch 39/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0379 - accuracy: 0.9870
Epoch 40/100
1688/1688 [==============================] - 14s 9ms/step - loss: 0.0319 - accuracy: 0.9889 0s - los
Epoch 41/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0359 - accuracy: 0.9874
Epoch 42/100
1688/1688 [==============================] - 1

1688/1688 [==============================] - 14s 8ms/step - loss: 0.1569 - accuracy: 0.9412
Epoch 10/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1444 - accuracy: 0.9451
Epoch 11/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1363 - accuracy: 0.9483
Epoch 12/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1256 - accuracy: 0.9524
Epoch 13/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1158 - accuracy: 0.9560
Epoch 14/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1056 - accuracy: 0.9602
Epoch 15/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0988 - accuracy: 0.9621
Epoch 16/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0920 - accuracy: 0.9652
Epoch 17/100
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0855 - accuracy: 0.9677
Epoch 18/100
1688/1688 [==============================] - 14s 8ms/st

1688/1688 [==============================] - 15s 9ms/step - loss: 0.0278 - accuracy: 0.9918
Epoch 87/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0287 - accuracy: 0.9913
Epoch 88/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0235 - accuracy: 0.9929
Epoch 89/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0227 - accuracy: 0.9935
Epoch 90/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0238 - accuracy: 0.9928
Epoch 91/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0292 - accuracy: 0.9920
Epoch 92/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0244 - accuracy: 0.9929
Epoch 93/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0240 - accuracy: 0.9928
Epoch 94/100
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0256 - accuracy: 0.9922
Epoch 95/100
1688/1688 [==============================] - 15s 9ms/st

0.9038166642189026